In [0]:

from pyspark.sql.functions import schema_of_json, from_json, col

# Sample JSON (can come from a file or Kafka message)
sample_json = """
{"customer_id": 9179, "customer_name": "Richard Cox", "date_of_birth": "1996-10-25", "telephone": "+1 6680703335", "email": "devon84@mail.com", "member_since": "2024-09-26", "created_timestamp": "2024-10-17 16:12:27"}
"""

# Infer schema from the sample
json_schema = spark.range(1).select(schema_of_json(sample_json).alias("schema")).collect()[0]["schema"]
print(json_schema)


In [0]:

customers_df = (
                    spark.readStream
                         .format("json")
                         .schema(json_schema)
                         .load("/Volumes/gizmobox/landing/operational_data/customers_stream/")
)

In [0]:
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (
                                customers_df.withColumn("file_path", col("_metadata.file_path"))
                                            .withColumn("ingestion_date", current_timestamp())
)

In [0]:
streaming_query = (
    customers_transformed_df.writeStream
        .format("delta")
        .outputMode("append")
        .trigger(processingTime='10 seconds')
        .option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream")
        .toTable("gizmobox.bronze.customers_stream")
)

In [0]:
streaming_query.stop()

In [0]:
%sql
select * from gizmobox.bronze.customers_stream
